In [1]:
import pandas as pd
import numpy as np
from io import StringIO

In [33]:
df = pd.read_csv("..\Corpora\ICE INDIA\Corpus\S1a-001.txt", sep="<$A>|<$B>", engine='python')


In [30]:
pip install -U scikit-learn

     ---------------------------------------- 8.3/8.3 MB 433.4 kB/s eta 0:00:00
     ------------------------------------ 298.0/298.0 kB 735.9 kB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
df = df.replace(to_replace=r'<.*>', value='', regex=True)
print(df)

                           <I>
0                             
1                             
2    May I know your good name
3                             
4                             
..                         ...
711                           
712                 Thank you 
713                           
714                           
715       Thank you very much 

[716 rows x 1 columns]


In [6]:
padded_numbers = [number.zfill(3) for number in numbers]
print(padd_numbers)

NameError: name 'numbers' is not defined

In [44]:
#Indian Corpus
count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
comb=''
counter=0
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE INDIA\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<[^<>]*>', value='', regex=True)
        temp_df = temp_df.apply(lambda temp_df: temp_df.str.lower())
        #comb = comb.append(temp_df)
        new_filepath = "..\CleanCorpora\India\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False, header=False)
        temp_df = temp_df.apply(lambda x: ' '.join(x.astype(str)), axis=1)
        temp_df = temp_df.replace("", np.nan)
        temp_df = temp_df.replace("\s+", " ", regex=True).dropna()
        comb = ' '.join(temp_df)
        comb_filepath = "..\CleanCorpora\India\_IndianCorporaCombined{0}.txt".format(counter)
        with open(comb_filepath, 'w',encoding='UTF-8') as f:
            f.write(comb)
        counter+=1
#combDF = pd.DataFrame(eval(comb))


#combDF.to_csv(comb_filepath, sep='\t', index=False)

In [46]:
dfs = []
for i in range(300):
    #df = pd.read_csv("..\CleanCorpora\India\_IndianCorporaCombined{0}.txt".format(i), sep="\t", engine="python", encoding="UTF-8")
    with open("..\CleanCorpora\\India\_IndianCorporaCombined{0}.txt".format(i), "r", encoding='UTF-8') as file:
        text = file.read()
    df = pd.DataFrame([text],columns=[" "])
    dfs.append(df)
    #combdf = pd.concat([combdf, df], ignore_index=True)
comb = pd.concat(dfs, ignore_index=True)
print(comb.shape)
print(comb.iloc[0, 0])
comb.to_csv("..\CleanCorpora\India\__FULLIndianCorporaCombined.csv", sep='\t', index=False, header=False)
    #with open("..\CleanCorpora\_IndianCorporaCombined{0}.txt".format(i), "r", encoding='UTF-8') as file:
        #text = file.read()

(300, 1)
may i know your good name my name is mrs lalita   i'm from madras i see   i'm adeline fernandes coming from goa ah i belong to the diocese congregration of one word of christ  one or two words   ah   i'm just come here on a holiday  so  speak  so how is the weather here sister how do you feel the weather here i feel very hot very hot yeah it makes you very tiresome ahm you know  ahm   one word  somehow like no it's very hot like in the afternoon only it is burning the skin is burning like whereas in goa we have the cool climate even in madras also we have na uh sea-shore and after three o'clock uh in the afternoon we have a very cool breeze and we enjoy that sea breeze like anything  clears throat  we like it yeah this is very hot i don't know how these people uh carry here you know they can't help it no this is but in the afternoon you know uhm  they go out  it's like burning i don't know how it is like laugh and you come home you feel that's a day  uhm  madras breeze is a ve

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from io import open

In [56]:
# Open the text file
with open("..\CleanCorpora\India\__FULLIndianCorporaCombined.txt", "r", encoding='ISO-8859-1') as file:
    text = file.read()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([text])
print(X.shape)
feature_names = vectorizer.get_feature_names()
print(feature_names) 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X, target)
target_pred = clf.predict(X)
acc = accuracy_score(target, target_pred)
#Need to work out how to maintain the label vectors
#Before this I think I need to apply the cleaning process to the other corpora
#Also look back at the AI CW 1 to help remember how the tfidf works and how this is inputted with the label data for training
#Maybe also create the training data as well whie you're at it

(1, 23477)
['00', '015', '1966', '25', '8th', '91', 'aa', 'aaa', 'aad', 'aadam', 'aadat', 'aadhar', 'aadmi', 'aage', 'aahariya', 'aai', 'aaichi', 'aaichich', 'aaina', 'aaj', 'aajeev', 'aakal', 'aakals', 'aala', 'aam', 'aamir', 'aangika', 'aangikabhinaya', 'aap', 'aapane', 'aapke', 'aapko', 'aapne', 'aarti', 'aasana', 'aashirwad', 'aata', 'aatakathas', 'aatam', 'aatohari', 'aau', 'aaya', 'aaye', 'aayei', 'ab', 'abadi', 'abandon', 'abandoned', 'abandonment', 'abandons', 'abars', 'abba', 'abbas', 'abdomen', 'abdominal', 'abducted', 'abdul', 'abdulla', 'abdullah', 'abdus', 'abedience', 'abegging', 'aberrations', 'abeyance', 'abhay', 'abhi', 'abhinav', 'abhinava', 'abhinaya', 'abhinayam', 'abhipraya', 'abhiyan', 'abhor', 'abhors', 'abhudaya', 'abide', 'abiding', 'abilities', 'ability', 'able', 'abnormal', 'abolish', 'abolished', 'abolishing', 'abolition', 'abolutely', 'aboriginal', 'abortion', 'abortive', 'abot', 'about', 'above', 'abraham', 'abreast', 'abridges', 'abroad', 'abrupt', 'abrup

C:\Users\iwanr\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [1, 23477]

In [57]:
print(X)
print(len(target))

  (0, 10296)	1.6536507548014483e-05
  (0, 8595)	1.6536507548014483e-05
  (0, 20710)	1.6536507548014483e-05
  (0, 12335)	1.6536507548014483e-05
  (0, 12337)	1.6536507548014483e-05
  (0, 8220)	1.6536507548014483e-05
  (0, 22844)	4.9609522644043446e-05
  (0, 22843)	1.6536507548014483e-05
  (0, 8101)	1.6536507548014483e-05
  (0, 18559)	1.6536507548014483e-05
  (0, 779)	1.6536507548014483e-05
  (0, 17420)	1.6536507548014483e-05
  (0, 9571)	6.614603019205793e-05
  (0, 6107)	1.6536507548014483e-05
  (0, 3138)	1.6536507548014483e-05
  (0, 9652)	4.9609522644043446e-05
  (0, 8300)	1.6536507548014483e-05
  (0, 22793)	1.6536507548014483e-05
  (0, 11381)	1.6536507548014483e-05
  (0, 23472)	1.6536507548014483e-05
  (0, 19864)	1.6536507548014483e-05
  (0, 14160)	1.6536507548014483e-05
  (0, 19863)	1.6536507548014483e-05
  (0, 7180)	1.6536507548014483e-05
  (0, 15197)	1.6536507548014483e-05
  :	:
  (0, 23239)	0.02910425328450549
  (0, 14577)	0.08764349000447676
  (0, 14518)	0.28470905045416534
  (0, 4

In [38]:
count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
comb = pd.DataFrame()
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE-HK\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<[^<>]*>', value='', regex=True)
        temp_df = temp_df.replace("", np.nan)
        temp_df = temp_df.replace("\s+", " ", regex=True).dropna()
        temp_df = temp_df.apply(lambda temp_df: temp_df.str.lower())
        comb = comb.append(temp_df)
        new_filepath = "..\CleanCorpora\HongKong\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False)
comb_filepath = "..\CleanCorpora\HongKong\_HongKongCorporaCombined.txt"
comb.to_csv(comb_filepath, sep='\t', index=False)

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa

C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_23708\1126905689.py:15: FutureWa